SQL DATABASES 
NOTE: we have already done - NoSQL (Unstructured-JSON), Now we are doing the Structured ones(STructured)

In [5]:
import sqlite3
import os

os.makedirs('data/databases', exist_ok=True)

In [6]:
#Create sample database connection
conn=sqlite3.connect('data/databases/company.db')
cursor=conn.cursor()

In [7]:
#Create Tables
cursor.execute(
    '''CREATE TABLE IF NOT EXISTS employees (id INTEGER PRIMARY KEY, name TEXT, role TEXT, department TEXT, salary REAL)'''
)

In [8]:
cursor.execute('''CREATE TABLE IF NOT EXISTS projects (id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget REAL, lead_id INTEGER)''')

In [9]:
# Insert sample data
employees = [
    (1, 'John Doe', 'Senior Developer', 'Engineering', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000),
    (3, 'Mike Johnson', 'Product Manager', 'Product', 110000),
    (4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline', 'Completed', 80000, 2),
    (3, 'Customer Portal', 'Planning', 200000, 3),
    (4, 'ML Platform', 'Active', 250000, 2)
]


In [10]:
cursor.executemany('INSERT OR REPLACE INTO employees VALUES (?,?,?,?,?)', employees)
cursor.executemany('INSERT OR REPLACE INTO projects VALUES (?,?,?,?,?)', projects)

In [11]:
cursor.execute("Select * from employees")

In [12]:
conn.commit()
conn.close()

# DataBase Content Extraction

In [13]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

In [14]:
## Method1 - SQLDATABASE Utility
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

## get Database info
print(f"Tables: {db.get_usable_table_names()}")
print(f"\nTable DDL: ")
print(db.get_table_info())


Tables: ['employees', 'projects']

Table DDL: 

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	John Doe	Senior Developer	Engineering	95000.0
2	Jane Smith	Data Scientist	Analytics	105000.0
3	Mike Johnson	Product Manager	Product	110000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline	Completed	80000.0	2
3	Customer Portal	Planning	200000.0	3
*/


In [19]:
from typing import List
from langchain_core.documents import Document
#Method2: Custom SQL to Document conversion

print("\n Custom SQL Processing")

def sql_to_documents(db_path: str)->List[Document]:
    """ Convert SQL database to documents with context"""
    conn=sqlite3.connect(db_path)
    cursor=conn.cursor()
    documents=[]


    #Strategy 1: Create documents for each table
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'; ")
    tables=cursor.fetchall()

    for table in tables:
        table_name = table[0]

        #Get table schema
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        column_names=[col[1] for col in columns]

        #Get Table Data
        cursor.execute(f"SELECT * FROM {table_name}")
        rows=cursor.fetchall()

        #Create table overview document 
        table_content = "Sample Records:\n"
        for row in rows[:5]:
            record = dict(zip(column_names, row))
            table_content+=f"{record}\n"
        
        doc=Document(
            page_content=table_content,
            metadata={
                'source': db_path,
                'table_name': table_name,
                'num_records': len(rows),
                'data_type': 'sql_table'
            }
        )
        documents.append(doc)
    
    #create relationship documents
    #example: Joi employees and projects
    cursor.execute("""
    SELECT e.name, e.role, p.name as project_name, p.status
    FROM employees e
    JOIN projects p ON e.id=p.lead_id
    """)

    relationships=cursor.fetchall()
    print(relationships)
    rel_content="Employee-Project Relationships:\n\n"
    for rel in relationships:
        rel_content += f"{rel[0]} ({rel[1]}) leads {rel[2]} -status: {rel[3]}"

    rel_doc=Document(
        page_content=rel_content,
        metadata={
            'source': db_path,
            'data_type': 'sql_relationships',
            'query': 'employee_project_join'
        }
    )
    documents.append(rel_doc)

    conn.close()
    return documents


 Custom SQL Processing


In [20]:
sql_to_documents("data/databases/company.db")

[('John Doe', 'Senior Developer', 'RAG Implementation', 'Active'), ('Jane Smith', 'Data Scientist', 'Data Pipeline', 'Completed'), ('Mike Johnson', 'Product Manager', 'Customer Portal', 'Planning'), ('Jane Smith', 'Data Scientist', 'ML Platform', 'Active')]


[Document(metadata={'source': 'data/databases/company.db', 'table_name': 'employees', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Sample Records:\n{'id': 1, 'name': 'John Doe', 'role': 'Senior Developer', 'department': 'Engineering', 'salary': 95000.0}\n{'id': 2, 'name': 'Jane Smith', 'role': 'Data Scientist', 'department': 'Analytics', 'salary': 105000.0}\n{'id': 3, 'name': 'Mike Johnson', 'role': 'Product Manager', 'department': 'Product', 'salary': 110000.0}\n{'id': 4, 'name': 'Sarah Williams', 'role': 'DevOps Engineer', 'department': 'Engineering', 'salary': 98000.0}\n"),
 Document(metadata={'source': 'data/databases/company.db', 'table_name': 'projects', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Sample Records:\n{'id': 1, 'name': 'RAG Implementation', 'status': 'Active', 'budget': 150000.0, 'lead_id': 1}\n{'id': 2, 'name': 'Data Pipeline', 'status': 'Completed', 'budget': 80000.0, 'lead_id': 2}\n{'id': 3, 'name': 'Customer Portal', 'status': 'Planni